In [ ]:
%matplotlib inline

import sys
import json
from pathlib import Path

from PIL import Image
from tqdm.notebook import tqdm
import numpy as np
import pandas as pd
from torchvision import models as torchvision_models

from bananas.training.criteria import HaltCriteria
from bananas.dataset import DataSet, DataType, Feature

# Root path of project relative to this notebook
ROOT = Path('..')

sys.path.insert(1, str(ROOT / 'scripts'))
from datamodels import *
from utils import *

### Load pre-trained models to be used as starting point

In [ ]:
resnet34_model = torchvision_models.resnet34(pretrained=True)
googlenet_model = torchvision_models.googlenet(pretrained=True)

### Read subject data from local file

In [ ]:
df = pd.read_csv(ROOT / 'datasets' / 'subject_diagnosis.csv', index_col=0)

# Convert non-primitive fields
df['processed_path'] = df['processed_path'].apply(lambda x: Path(x))
df['image_path'] = df['image_path'].apply(lambda x: Path(x))
df['template_path'] = df['template_path'].apply(lambda x: Path(x))
df['drawing_box'] = df['drawing_box'].apply(lambda x: Box.load(x))
df['template_box'] = df['template_box'].apply(lambda x: Box.load(x))

# Remove all unnecessary columns from our dataset
feat_keys = ['processed_path']
group_columns = ['diagnosis']
df = df[group_columns + feat_keys].copy()

# Normalize all feature columns
df = df.dropna()
for col in feat_keys:
    df[col] = df[col].apply(lambda x: str(ROOT / x))

df.head()

### Create a file to hold intermediate results

In [ ]:
results_file = ROOT / 'results' / 'modelzoo_transfer.csv'

if not results_file.exists():
    pd.DataFrame(columns=[
        'Key',
        'Trial',
        'Model',
        'Subset splits', 
        'Batch size', 
        'Random seed', 
        'Δ naive classifier',
        'Accuracy',
        'Precision',
        'Recall',
        'Area under ROC']).set_index('Key').to_csv(results_file)

trial_results = pd.read_csv(results_file, index_col='Key')

### Train transfer learning model

In [ ]:
from itertools import product, combinations
    
# Define all possible hyperparameters
models = [resnet34_model, googlenet_model]
batch_sizes = [24, 32, 48, 56, 64]
test_splits = [.2, .25]
validation_splits = [.2, .25]

# Initialize random number generator without seed to randomize hyperparamters
rnd = np.random.RandomState()

# Cross product all hyperparameters
parameter_combinations = list(product(
    models, batch_sizes, test_splits, validation_splits))
rnd.shuffle(parameter_combinations)

target_label = 'SANO'
target_column = 'diagnosis'

In [ ]:
from bananas.core.mixins import HighDimensionalMixin
from bananas.sampling.cross_validation import DataSplit
from bananas.statistics import scoring
from bananas.statistics.scoring import ScoringFunction
from coconuts.learners.transfer_learning import TransferLearningModel, BaseNNClassifier

# Perform 3 trials per parameter set to later compute the average
trial_count = 3

for model, batch_size, test_split, validation_split in tqdm(parameter_combinations, leave=False):

    trial_params = {
        'Model': model.__class__.__name__,
        'Subset splits': (test_split, validation_split),
        'Batch size': batch_size,
    }
    
    # Execute the independent trials
    for trial_num in tqdm(range(trial_count), leave=False, desc='Trial'):
        trial_params['Trial'] = trial_num
        
        # If these parameters have already been tried, we can skip trial
        trial_key = '|'.join(['%s=%s' % (k, str(v)) for k, v in trial_params.items()])
        if trial_key in trial_results.index: continue
            
        # Create a single feature containing all image data
        image_loader = ImageAugmenterLoader(
            df['processed_path'].values,
            resize=(3, 224, 224),
            normalize=True,
            convert='RGB')
        features = [Feature(
            image_loader,
            kind=DataType.HIGH_DIMENSIOAL,
            sample_size=10,
            random_seed=0)]

        # Define target feature
        target_feature = Feature(
            (df[target_column] == target_label).values, random_seed=0)

        # Always re-initialize the randopm seed
        random_seed = trial_num

        while True:

            # Change seed at every iteration to ensure different dataset split
            random_seed = np.random.RandomState(seed=random_seed).randint(1E6)
            
            # Build dataset, making sure that we have a left-out validation subset
            dataset = DataSet(
                features,
                name=target_label,
                target=target_feature,
                random_seed=random_seed,
                batch_size=batch_size,
                test_split=test_split,
                validation_split=validation_split)

            # Compute test class balance to tell what minimum accuracy we should beat
            test_idx = dataset.sampler.subsamplers[DataSplit.VALIDATION].data
            test_classes = target_feature[test_idx]
            test_class_balance = sum(test_classes) / len(test_classes)

            # Rebuild dataset unless test class balance is within 5% of ground truth
            true_class_balance = sum(target_feature[:] / len(target_feature))
            if abs(test_class_balance - true_class_balance) < .05: break

        # Instantiate learner using pre-trained model
        learner = TransferLearningModel(
            model,
            freeze_base_model=True,
            scoring_function=ScoringFunction.ACCURACY) \
            .apply_mixin(BaseNNClassifier, HighDimensionalMixin)

        # Train learner using train dataset
        learner.train(dataset.input_fn, progress=True, max_steps=1000)

        # Test learner predictions using left-out dataset
        # We have to do it one datapoint at a time instead of in batch to prevent overflow
        yl, ylt = [], []
        for i in tqdm(test_idx, leave=False):
            X, y = dataset[i:i+1]
            y = learner.label_encoder_.transform(y)
            y_ = learner.predict_proba(X)
            yl.append(y[0])
            ylt.append(y_[0])
        y, y_ = yl, ylt
        score_auroc = scoring.score_auroc(y, y_)
        score_accuracy = scoring.score_accuracy(y, y_)
        score_precision = scoring.score_precision(y, y_)
        score_recall = scoring.score_recall(y, y_)

        # Store trial results
        naive_accuracy = max(test_class_balance, 1 - test_class_balance)
        trial_results.loc[trial_key] = {
            **trial_params,
            'Random seed': random_seed,
            'Δ Naive Classifier': score_accuracy - naive_accuracy,
            'Accuracy': score_accuracy,
            'Precision': score_precision,
            'Recall': score_recall,
            'Area under ROC': score_auroc,
        }
        trial_results = trial_results.sort_values('Accuracy', ascending=False)
        trial_results.to_csv(results_file)

In [ ]:
trial_results.head(15)